# Spatio-temporal Clip With top view and Bar plot
@author: Max Felius

In [3]:
#imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os, sys, time
import re, datetime
from scipy import spatial
from tqdm import tqdm

#import personal functions
sys.path.extend(os.path.join(os.getcwd(),'sinkhole_functions'))
from sinkhole_functions.functions import *
from sinkhole_functions.influence_functions import *
from sinkhole_functions.spatio_temporal_functions import *

In [4]:
folder = ''
filename1 = 'Implemented_linear_Franciscanerstraat_sinkhole_final.csv'
# filename1 = 'Subset_MRSS_s1_asc_t88_x0203050.00_y0318500.00_r500m.csv'
# filename1 = 'Sinkhole_Implemented_Dataset_3.csv'
# filename1 = 'Subset_MRSS_s1_asc_t88_x0_203050.00_y0_318500.00_r100m.csv'
location_file = 'sinkhole_kerkrade.csv'

#load the data
data = pd.read_csv(os.path.join(folder,filename1))
location = pd.read_csv(os.path.join(folder,location_file))

location_gdf = gpd.GeoDataFrame(location, geometry=gpd.points_from_xy(location.longitude, location.latitude))
location_gdf.crs = 'epsg:4326' #wgs84
location_gdf = location_gdf.to_crs('epsg:28992') # to Amersfoort Rijksdriehoekenstelsel

# #---------------------#
# #make the kdtree for quick subset creation
# rdx = data['pnt_rdx'].values
# rdy = data['pnt_rdy'].values
# rd_data = np.concatenate((rdx.reshape(len(rdx),1),rdy.reshape(len(rdy),1)),axis=1)
# tree = spatial.cKDTree(rd_data)

# x0 = location_gdf.geometry.x.values[0]
# y0 = location_gdf.geometry.y.values[0]

# #cropping the dataset
# subset = tree.query_ball_point(([x0,y0]),r=80)
# data = data.iloc[subset]
# #-------------------#


#get information from pandas df
headers = list(data)
data_gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.pnt_lon, data.pnt_lat))
data_gdf.crs = 'epsg:4326' #wgs84
data_gdf = data_gdf.to_crs('epsg:28992') #to Amersfoort Rijksdriehoekenstelsel


#select point locations
rdx = data_gdf.geometry.x.values
rdy = data_gdf.geometry.y.values

#make the kdtree for quick subset creation
rd_data = np.concatenate((rdx.reshape(len(rdx),1),rdy.reshape(len(rdy),1)),axis=1)
tree = spatial.cKDTree(rd_data)

In [5]:
R = 35
epoch_vector = np.array([10])

#obtain limits dataset
xmin = np.min(rdx)
xmax = np.max(rdx)
ymin = np.min(rdy)
ymax = np.max(rdy)

#Define the grid for evaluation
n = 40
x_len = xmax-xmin
y_len = ymax-ymin
x_range = np.linspace(xmin+(0.1*x_len),xmax-(0.1*x_len),n)
y_range = np.linspace(ymin+(0.1*y_len),ymax-(0.1*y_len),n)

xv, yv = np.meshgrid(x_range,y_range)
xv_u = xv.ravel()
yv_u = yv.ravel()

epochs = np.array(get_sentinel_days(list(data_gdf))) #get all the sentinel epochs

#save the data
save_folder = f'new_clip_spatioTemp_final_{n}x{n}_{epoch_vector[0]}n_R{R}'

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

In [6]:
(-xmin+xmax)/n

6.255507571720227

In [7]:
# epoch_vector = np.array([10,15,20])
epoch_xhat = []
epoch_qxhat = []

for idx,nepoch in enumerate(epoch_vector): #number of epochs used
    # setting up variables
    n_points = len(xv_u)
    n_time = nepoch
    #define the time window of a sinkhole
    steps = np.arange(0,len(epochs)-n_time,1)
    n_steps = len(steps)

    # pre allocate space for variable
    xhat_save = np.zeros((n_points,n_steps))
    qxhat_save = np.zeros((n_points,n_steps))
    fxhat_save = np.zeros((n_points,n_steps))
    fqxhat_save = np.zeros((n_points,n_steps))

    #start the grid-wise search
    for i in tqdm(range(n_points),f'{idx+1}/{len(epoch_vector)}'):
        #select new center point
        xi = xv_u[i]
        yi = yv_u[i]

        #select subset
        subset = tree.query_ball_point(([xi,yi]),r=R)

        #start test
        if len(subset) <= 1: 
            #subset is empty
            for iii in range(n_steps):
                xhat_save[i,iii] = np.nan
                qxhat_save[i,iii] = np.nan
        else:
            #compute distance towards center (radius)
            xp = data_gdf.geometry.x.values[subset]
            yp = data_gdf.geometry.y.values[subset]

            r = np.sqrt((xp-xi)**2 + (yp-yi)**2)

            for ii in steps:
                ii = int(ii)
                #get relative days
                selected_epochs = epochs[ii:ii+n_time]
                t, start_day = get_delta_days(selected_epochs)

                y = data_gdf[selected_epochs].iloc[subset].values

                #calculate subsidence velocity
                t_vector, y_vector, r_vector = create_matching_len(t,y,r)

                xhat, Qxhat, fit = kinematic_model(R,r_vector,y_vector,t_vector)

                #save the parameters
                xhat_save[i,ii] = xhat
                qxhat_save[i,ii] = Qxhat
                fqxhat_save[i,ii] = filter_extremes2(Qxhat,maxvalue=1e-4,minvalue=0)
#                 fxhat_save[i,ii] = filter_extremes(xhat,maxvalue=-0.0001,minvalue=-0.001)
                fxhat_save[i,ii] = filter_extremes(xhat,maxvalue=0,minvalue=-0.0006)
    
    epoch_xhat.append(fxhat_save)
    epoch_qxhat.append(fqxhat_save)

1/1: 100%|██████████| 1600/1600 [08:46<00:00,  3.04it/s]


In [8]:
#set parameters
cmin =0
cmax =0.0006
start= 80
number_text=0.0001 #show only values above this threshold

#import image
img = plt.imread('background.png')
extents = (xmin, xmax, ymin, ymax)

#convert epochs into datetime object
epoch_sel = np.array(list(map(lambda e: datetime.datetime.strptime(e,'d_%Y%m%d'),epochs)))

#defining epochs used
epoch_range = np.arange(epoch_vector[0]+start,len(epoch_sel)-epoch_vector[0],1)

for epoch_selec in epoch_range:
    fig, ax = plt.subplots(ncols=2,figsize=(15,10))

    fig.suptitle(f'Epoch selected: {epoch_sel[epoch_selec+epoch_vector[0]]}')
    
    #apply filters
    number = 0.0001 #filter number 2
#     number = 0
    filter_epoch_values = -fxhat_save[:,epoch_selec]
    a = filter_epoch_values>number
    b = fqxhat_save[:,epoch_selec] > 0
    color_grid = [i and j for i, j in zip(a,b)]
    
    #---------first plot--------------#
    ax[0].imshow(img,extent=extents)
    ax[0].scatter(rdx,rdy,c='r',s=5,label='Observations')
    
#     color_grid = -fxhat_save[:,epoch_selec]

#     number = 0
    h = ax[0].scatter(xv_u[color_grid],yv_u[color_grid],c=filter_epoch_values[color_grid],vmin=cmin,vmax=cmax)#,s=70)
    plt.colorbar(h,ax=ax[0])
    
    for idx,item in enumerate(color_grid):
        if item:
#     numbers = range(len(xv_u))
#             ax[0].text(xv_u[idx],yv_u[idx],s=f'{idx}')
            pass
    
    #admin stuff
    ax[0].set_xlabel('Distance East [m]')
    ax[0].set_ylabel('Distance North [m]')
    ax[0].legend()
    
    #---------second plot--------------#
    for idx,values in enumerate(zip(range(len(xv_u)),filter_epoch_values)):
        i,j = values
        if color_grid[idx]:
            ax[1].bar(i,j,color='b')#filter_epoch_values[idx],vmin=cmin,vmax=cmax)
            if j>number_text:
    #             print(j,number,j>number)
#                 ax[1].text(idx,j,s=f'{idx}')
                pass
    
#     ax[1].bar(range(len(xv_u)),filter_epoch_values)
    
#     for idx,item in enumerate(filter_epoch_values[color_grid]):
#         if item>number:
#             ax[1].text(idx,item,s=f'{idx}')
       
    
    #admin stuff
    ax[1].set_ylim([cmin,cmax])
    ax[1].set_xlabel('Point ID')
    ax[1].set_ylabel('Subsidence Velocity [m/day]')
    ax[1].set_xlim([0,len(xv_u)])
    ax[1].grid(True)
    
#     break
    plt.savefig(os.path.join(save_folder,'{:02d}_{}.png'.format(epoch_selec,epoch_sel[epoch_selec+epoch_vector[0]].strftime('%Y-%m-%d'))))
    plt.close('all')

## How to create a vide
`ffmpeg -framerate 2 -pattern_type glob -i '*.png' -vcodec libx264 -pix_fmt yuv420p -vb 20M spatio_temp_method.mp4`

In [47]:
epoch_sel[epoch_selec+epoch_vector[0]].strftime('%Y-%m-%d')

'2018-12-31'